## import

In [ ]:
pip install name_matching

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.6 MB/s eta 0:00:00


In [ ]:
pip install rapidfuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 27.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

from dateutil import parser
import datetime
from datetime import datetime
from datetime import time
import matplotlib.pyplot as plt
import unicodedata
import re
from tqdm import tqdm
from cleanco import basename
import ast

In [ ]:
from rapidfuzz import fuzz, utils
from rapidfuzz import process, fuzz

# Data

In [ ]:
# Preqin
Preqin_deals = pd.read_excel("/content/drive/MyDrive/UT_research/VC_IB/data/preqin/Preqin_deals_1983_2023.xlsx")
Preqin_deals.shape

In [ ]:
sdc_ipo_20230930 = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/sdc_ipo_20230930.csv')
sdc_ipo_20230930.drop('Unnamed: 0',axis=1,inplace=True)
sdc_ipo_20230930

,jkl_index,deal_number,filing_date,issue_date,issuer,main_sic_code,nation,lead_underwriter_code,all_managers,all_mgr_role_code,...,Close_Price,initial_return,factor_2022,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed
0,5,4.529140e+08,1994-09-19,1995-01-10,DiaSys Corp,3841,UnitedStates,TASIN,Tasin & Company,BM,...,5.6875,0.137500,1.732604,8.663020,Within range,5.0,5.0,5.0,5.0,No
1,21,4.464180e+08,1994-10-20,1995-01-12,SunPharm Corp,2834,UnitedStates,ROYCE-INVEST,Royce Investment Group Inc,BM,...,7.5000,0.071429,1.732604,13.341051,Within range,7.0,9.0,7.0,9.0,No
2,29,4.455930e+08,1994-10-13,1995-01-18,California Pro Sports Inc,5091,UnitedStates,COHIG,Cohig & Associates,BM,...,4.5000,0.000000,1.732604,9.356062,Within range,4.5,5.5,4.5,5.5,No
3,30,4.460520e+08,1994-09-14,1995-01-18,Infosafe Systems Inc,7372,UnitedStates,D-H-BLAIR,D. H. Blair,BM,...,5.2500,0.050000,1.732604,13.514311,Within range,5.0,5.0,5.0,5.0,Yes
4,32,4.463820e+08,1994-10-18,1995-01-18,Timeline Inc,7372,UnitedStates,T-JAMES,Thomas James Associates,BM,...,4.2500,-0.150000,1.732604,8.663020,Within range,5.0,6.0,5.0,6.0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5994,221800,3.989232e+09,2022-10-17,2022-11-14,Acrivon Therapeutics Inc,8731,UnitedStates,MS/JEFFERIES-L,Morgan Stanley & Co,JB,...,16.6400,0.331200,1.000000,94.400000,Below range,16.0,18.0,16.0,18.0,No
5995,222075,4.015683e+09,NaN,2022-12-06,Adamas One Corp,5094,UnitedStates,NOTAPP,Not Applicable,BM,...,4.7400,0.053333,1.000000,20.100000,Within range,4.5,5.0,4.5,5.0,No
5996,222077,3.928598e+09,2022-05-31,2022-12-06,Adamas One Corp,5094,UnitedStates,ALEXANDER-LP,Alexander Capital LP,BM,...,4.7400,0.053333,1.000000,11.000000,Within range,4.5,5.0,4.5,5.0,No
5997,222256,3.956107e+09,2022-07-29,2022-12-19,Lipella Pharmaceuticals Inc,2836,UnitedStates,SPARTAN-CAPITA,Spartan Capital Securities LL,BM,...,4.7100,-0.180870,1.000000,16.800000,Within range,5.0,7.0,5.0,7.0,No


In [ ]:
# drop at least one is missing
drop_list = ['PORTFOLIO COMPANY ID', 'INVESTORS / BUYERS (FIRMS)','DEAL DATE','PORTFOLIO COMPANY']
Preqin_deals = Preqin_deals.dropna(subset=drop_list)
Preqin_deals["jkl_index2"] = Preqin_deals.index
Preqin_deals

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,PORTFOLIO COMPANY STATE/ COUNTY,ENTRY REVENUE MULTIPLE,PORTFOLIO COMPANY STATUS,BOUGHT FROM / SELLERS (FIRMS),INVESTORS / BUYERS (FUNDS),LEAD PARTNERS,ACQUIRED SHARE (%),FINANCIAL ADVISORS - BUYERS,FINANCIAL ADVISORS - SELLERS,jkl_index2
0,V143231,56414,10380,Bessemer Venture Partners,Private Equity Firm (Investor),US,1983-01-01,Completed,Unspecified Round,"Maxim Integrated Products, Inc.",...,CA,NaN,Realised,NaN,NaN,NaN,NaN,NaN,NaN,0
1,V113205,216808,395,Institutional Venture Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,1
2,V113205,216808,400,InterWest Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,2
3,V113205,216808,486,Matrix Partners,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,3
4,V113205,216808,505,Merrill Pickard Anderson & Eyre,Private Equity Firm,US,1983-03-14,Completed,Unspecified Round,Bridge Communications Inc.,...,CA,NaN,Active,NaN,"InterWest Partners II, Warburg Pincus Associates",NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738450,V356543,358306,351600,UK Innovation & Science Seed Fund,Investment Company,UK,2023-09-28,Completed,Seed,Pencil Biosciences Limited,...,NaN,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,738450
738451,V356543,358306,444399,Northern Gritstone,Private Equity Firm,UK,2023-09-28,Completed,Seed,Pencil Biosciences Limited,...,NaN,NaN,Active,NaN,NaN,Duncan Johnson,NaN,NaN,NaN,738451
738452,V356550,112589,9403,Softbank China Venture Capital,Private Equity Firm (Investor),China,2023-09-28,Completed,Series D,Hydrexia Pty Ltd,...,NSW,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,738452
738453,V352746,477139,412866,SEE Fund,Other,China,2023-09-30,Completed,Unspecified Round,"Suzhou Fumailei Medical Technology Co., Ltd.",...,Jiangsu,NaN,Active,NaN,NaN,Jing He,NaN,NaN,NaN,738453


# After Rapid fuzz

## data

In [ ]:
preqin_sdc_fuzzmatch = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_results/preqin_sdc_fuzzmatched_1024.csv')
preqin_sdc_fuzzmatch = preqin_sdc_fuzzmatch.drop('Unnamed: 0',axis=1)
preqin_sdc_fuzzmatch

,jkl_index,jkl_index2,sdc_name,preqin_name,score,proccessed_sdc_name,proccessed_preqin_name
0,2999,943,Sequoia Software Corp,Sequoia Software,100.000000,sequoia software,sequoia software
1,3944,3983,Shutterfly Inc,Shutterfly LLC,100.000000,shutterfly,shutterfly
2,3947,4924,eHealth Inc,eHealth Inc,100.000000,ehealth,ehealth
3,3957,6996,Achillion Pharmaceuticals Inc,Achillion Pharmaceuticals Inc,100.000000,achillion pharmaceuticals,achillion pharmaceuticals
4,3962,39408,ORBCOMM Inc,ORBCOMM Inc,100.000000,orbcomm,orbcomm
...,...,...,...,...,...,...,...
5994,3922,382615,"Home BancShares Inc,Conway,AR",The Rainforest Company GmbH,53.061224,home bancshares incconwayar,the rainforest company
5995,2946,630488,Krispy Kreme Doughnuts Inc,Kriya RevGEN LLC,52.941176,krispy kreme doughnuts,kriya revgen
5996,1802,88790,"VRB Bancorp, Rogue River, OR",Urban Revivo,52.631579,vrb bancorp rogue river or,urban revivo
5997,1514,288029,"Flagstar Bancorp,Bloomfield,MI",Flare Star Game,51.162791,flagstar bancorpbloomfieldmi,flare star game


In [ ]:
fuzzmatch_additional = pd.read_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_results/preqin_sdc_fuzzmatched_1024_additional_matching.csv')
fuzzmatch_additional = fuzzmatch_additional.drop('Unnamed: 0',axis=1)
fuzzmatch_additional

## get the preqin data for matched deals

In [ ]:
# select only the perfect matches
preqin_sdc_fuzzmatch_100 = preqin_sdc_fuzzmatch[preqin_sdc_fuzzmatch['score'] >97.5]
# add the additional matches
preqin_sdc_fuzzmatch_100 = pd.concat([preqin_sdc_fuzzmatch_100,fuzzmatch_additional],axis=0)
preqin_sdc_fuzzmatch_100

,jkl_index,jkl_index2,sdc_name,preqin_name,score,proccessed_sdc_name,proccessed_preqin_name
0,2999,943,Sequoia Software Corp,Sequoia Software,100.000000,sequoia software,sequoia software
1,3944,3983,Shutterfly Inc,Shutterfly LLC,100.000000,shutterfly,shutterfly
2,3947,4924,eHealth Inc,eHealth Inc,100.000000,ehealth,ehealth
3,3957,6996,Achillion Pharmaceuticals Inc,Achillion Pharmaceuticals Inc,100.000000,achillion pharmaceuticals,achillion pharmaceuticals
4,3962,39408,ORBCOMM Inc,ORBCOMM Inc,100.000000,orbcomm,orbcomm
...,...,...,...,...,...,...,...
16,889,2555,Open Market Inc,OpenMarket,95.238095,open market,openmarket
17,4931,125719,DAVIDsTea Inc,Davids Tea,94.736842,davidstea,davids tea
18,422,295217,DataWorks Corp,Data Works,94.736842,dataworks,data works
19,3896,2780,Omrix Biopharmaceuticals Inc,Omrix Pharmaceuticals,93.333333,omrix biopharmaceuticals,omrix pharmaceuticals


In [ ]:
# get the matched jkl_index2 and then get the portfolio companies
Preqin_deals_m1 = Preqin_deals[Preqin_deals['jkl_index2'].isin(preqin_sdc_fuzzmatch_100['jkl_index2'])]
# get the data of the matched portfolio companies
Preqin_deals_m2 = Preqin_deals[Preqin_deals['PORTFOLIO COMPANY ID'].isin(Preqin_deals_m1['PORTFOLIO COMPANY ID'])]
Preqin_deals_m2

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,PORTFOLIO COMPANY STATE/ COUNTY,ENTRY REVENUE MULTIPLE,PORTFOLIO COMPANY STATUS,BOUGHT FROM / SELLERS (FIRMS),INVESTORS / BUYERS (FUNDS),LEAD PARTNERS,ACQUIRED SHARE (%),FINANCIAL ADVISORS - BUYERS,FINANCIAL ADVISORS - SELLERS,jkl_index2
39,V82966,40987,117,Boston Millennia Partners,Private Equity Firm (Investor),US,1986-06-01,Completed,Unspecified Round,PAREXEL International Corporation,...,MA,NaN,Realised,NaN,Boston Capital Ventures International,NaN,NaN,NaN,NaN,39
74,V90085,73062,10380,Bessemer Venture Partners,Private Equity Firm (Investor),US,1988-06-01,Completed,Unspecified Round,Echelon,...,CA,NaN,Realised,NaN,NaN,NaN,NaN,NaN,NaN,74
88,V204323,38007,482,Marquette Venture Partners,Private Equity Firm,US,1989-04-04,Completed,Seed,"Stericycle, Inc.",...,IL,NaN,Active,NaN,Marquette Venture I,NaN,NaN,NaN,NaN,88
89,V204323,38007,295571,State Farm Ventures,Private Equity Firm,US,1989-04-04,Completed,Seed,"Stericycle, Inc.",...,IL,NaN,Active,NaN,Marquette Venture I,NaN,NaN,NaN,NaN,89
110,V23055,59381,173,CID Capital,Private Equity Firm,US,1990-06-01,Completed,Unspecified Round,"Software Artistry, Inc.",...,IN,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
734441,V353773,384603,383180,Paradigm Biocapital Advisors,Hedge Fund Manager,US,2023-09-05,Completed,PIPE,"Olema Pharmaceuticals, Inc.",...,CA,NaN,Active,NaN,"Lightspeed Opportunity Fund II, Vivo Opportuni...",NaN,NaN,NaN,NaN,734441
734442,V353773,384603,429127,Deep Track Capital,Fund Manager,US,2023-09-05,Completed,PIPE,"Olema Pharmaceuticals, Inc.",...,CA,NaN,Active,NaN,"Lightspeed Opportunity Fund II, Vivo Opportuni...",NaN,NaN,NaN,NaN,734442
734460,V353785,384603,31447,Silicon Valley Bank,Bank,US,2023-09-05,Completed,Venture Debt,"Olema Pharmaceuticals, Inc.",...,CA,NaN,Active,NaN,NaN,NaN,NaN,NaN,NaN,734460
735476,V354377,77187,21149,Hercules Capital,Private Equity Firm,US,2023-09-11,Completed,Venture Debt,"Senseonics Holdings, Inc.",...,MD,NaN,Realised,NaN,NaN,R. Bryan Jadot,NaN,NaN,NaN,735476


In [ ]:
# get the merge index from the preqin_sdc_fuzzmatch_100 results
Preqin_deals_m3 = pd.merge(Preqin_deals_m2, preqin_sdc_fuzzmatch_100, how='left',on = "jkl_index2")
Preqin_deals_m3

In [ ]:
# sort to ffill
Preqin_deals_m3 = Preqin_deals_m3.sort_values(by=['PORTFOLIO COMPANY ID','jkl_index2','jkl_index'])
# ffil the missing data
Preqin_deals_m3.loc[:,['jkl_index2',	'jkl_index',	'sdc_name',	'preqin_name',	'score',	'proccessed_sdc_name',	'proccessed_preqin_name']] =  Preqin_deals_m3.loc[:,['jkl_index2',	'jkl_index',	'sdc_name',	'preqin_name',	'score',	'proccessed_sdc_name',	'proccessed_preqin_name']].fillna(method='ffill')
# change the jkl index type to int
Preqin_deals_m3['jkl_index'] = Preqin_deals_m3['jkl_index'].astype('int')
Preqin_deals_m3

## matched preqin and SDC merge

In [ ]:
# made a mistake with the jkl_index. make another one with jkl_index3
Preqin_deals_m3.rename(columns = {'jkl_index' : 'jkl_index3'}, inplace = True)
sdc_ipo_20230930['jkl_index3'] = sdc_ipo_20230930.index

In [ ]:
# change str to list
sdc_ipo_20230930['lead_underwriter_list'] = sdc_ipo_20230930['lead_underwriter_list'].apply(ast.literal_eval)
sdc_ipo_20230930['JB_JL_jointlead'] = sdc_ipo_20230930['JB_JL_jointlead'].apply(ast.literal_eval)
sdc_ipo_20230930['CM_CL_comanager'] = sdc_ipo_20230930['CM_CL_comanager'].apply(ast.literal_eval)

In [ ]:
preqin_sdc_merged_1024 = pd.merge(Preqin_deals_m3,sdc_ipo_20230930, how='left',on = "jkl_index3")
preqin_sdc_merged_1024

,DEAL ID,PORTFOLIO COMPANY ID,INVESTOR ID,INVESTORS / BUYERS (FIRMS),INVESTOR TYPE,INVESTOR COUNTRY,DEAL DATE,DEAL STATUS,STAGE,PORTFOLIO COMPANY,...,Close_Price,initial_return,factor_2022,Proceeds_2022,Issue_priced_range,constructed_low,constructed_high,min_range,max_range,venture_backed
0,V14233,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1997-08-01,Completed,Seed,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
1,V14235,25649,231,Domain Associates,Private Equity Firm,US,1998-07-01,Completed,Series B,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
2,V14237,25649,231,Domain Associates,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
3,V14237,25649,430,Kleiner Perkins,Private Equity Firm (Investor),US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
4,V14237,25649,2378,QuestMark Partners,Private Equity Firm,US,1999-10-01,Completed,Series C,"Align Technology, Inc.",...,17.3125,0.331731,1.559630,202.751890,Below range,14.0,16.0,14.0,16.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19643,V345814,554599,63108,Woori Venture Partners,Private Equity Firm,South Korea,2002-06-01,Completed,Unspecified Round,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19644,V345821,554599,192019,Seoul Technology Investment,Private Equity Firm,South Korea,2012-06-01,Completed,Secondary Stock Purchase,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19645,V345826,554599,91892,Yuanta Investment,Private Equity Firm,South Korea,2017-09-25,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No
19646,V345829,554599,344805,G.N. Tech Venture Capital,Private Equity Firm,South Korea,2017-12-21,Completed,PIPE,"S&D Co., Ltd.",...,4.7500,0.055556,1.668019,10.842122,Below range,5.0,6.0,5.0,6.0,No


In [ ]:
# preqin_sdc_merged_1024.to_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/preqin_sdc_merged_1024.csv')

# link preqin investor and sdc underwiter

In [ ]:
investor_df = preqin_sdc_merged_1024.loc[:,['INVESTORS / BUYERS (FIRMS)','lead_underwriter_list','CM_CL_comanager']].copy()
investor_df['p_s_index1'] = investor_df.index
investor_df

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager,p_s_index1
0,Kleiner Perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",0
1,Domain Associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",1
2,Domain Associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",2
3,Kleiner Perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",3
4,QuestMark Partners,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",4
...,...,...,...,...
19643,Woori Venture Partners,"[Shamus Group, Inc]",[],19643
19644,Seoul Technology Investment,"[Shamus Group, Inc]",[],19644
19645,Yuanta Investment,"[Shamus Group, Inc]",[],19645
19646,G.N. Tech Venture Capital,"[Shamus Group, Inc]",[],19646


In [ ]:
# preproccess again to match the investment banks

suffix_2 = [' co',' inc',' llc',' corp',' ltd',' lp']
# mannually preproccess the company names
def preprocceser(text_1):
  # removing all capital letters.v
  text_1 = text_1.lower()
  # replace non-ASCII characters.
  text_1 = unicodedata.normalize('NFKD', text_1 ).encode('ASCII', 'ignore').decode()
  # remove any character that is not a word or space character with nothing
  text_1 = re.sub(r'[^\w\s]','',text_1)
  # remove common legal business suffixes and retun
  for word in suffix_2:
        text_1 = re.sub(fr"\b{word}\b", '', text_1)
  return basename(text_1)

In [ ]:
# preproccess first
investor_df['INVESTORS / BUYERS (FIRMS)'] = investor_df['INVESTORS / BUYERS (FIRMS)'].apply(preprocceser)

In [ ]:
# use the rapid fuzzywuzzy for lead underwriter matching
tempt_list = []
for i, comp in enumerate(tqdm(investor_df['INVESTORS / BUYERS (FIRMS)'])):
  tempt_list.append(process.extractOne(comp, [preprocceser(c) for c in investor_df['lead_underwriter_list'].iloc[i]], scorer=fuzz.WRatio))

100%|██████████| 19648/19648 [00:04<00:00, 4068.62it/s]


In [ ]:
# make data frame using the
luw_matched_df = pd.DataFrame(tempt_list).iloc[:,:2]
luw_matched_df = luw_matched_df.rename(columns={0: "matched_sdc_luwriter", 1:"l_underwriter_match_score", 2:'preqin_location'})
luw_matched_df

,matched_sdc_luwriter,l_underwriter_match_score
0,deutsche banc alex brown,37.500000
1,deutsche banc alex brown,29.268293
2,deutsche banc alex brown,29.268293
3,deutsche banc alex brown,37.500000
4,deutsche banc alex brown,33.333333
...,...,...
19643,shamus group,25.650000
19644,shamus group,33.750000
19645,shamus group,19.655172
19646,shamus group,23.478261


In [ ]:
# merge with the investor_df data
investor_df2 = pd.concat([investor_df, luw_matched_df],axis=1)
investor_df2

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager,p_s_index1,matched_sdc_luwriter,l_underwriter_match_score
0,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",0,deutsche banc alex brown,37.500000
1,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",1,deutsche banc alex brown,29.268293
2,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",2,deutsche banc alex brown,29.268293
3,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",3,deutsche banc alex brown,37.500000
4,questmark partners,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",4,deutsche banc alex brown,33.333333
...,...,...,...,...,...,...
19643,woori venture partners,"[Shamus Group, Inc]",[],19643,shamus group,25.650000
19644,seoul technology investment,"[Shamus Group, Inc]",[],19644,shamus group,33.750000
19645,yuanta investment,"[Shamus Group, Inc]",[],19645,shamus group,19.655172
19646,gn tech venture capital,"[Shamus Group, Inc]",[],19646,shamus group,23.478261


In [ ]:
# again use the rapid fuzzywuzzy for co underwriter matching
tempt_list2 = []
for i, comp in enumerate(tqdm(investor_df['INVESTORS / BUYERS (FIRMS)'])):
  tempt_list2.append(process.extractOne(comp, [preprocceser(c) for c in investor_df['CM_CL_comanager'].iloc[i]], scorer=fuzz.WRatio))

100%|██████████| 19648/19648 [00:04<00:00, 4259.23it/s]


In [ ]:
# make data frame using the
couw_matched_df = pd.DataFrame(tempt_list2).iloc[:,:2]
couw_matched_df = couw_matched_df.rename(columns={0: "matched_sdc_couwriter", 1:"co_underwriter_match_score", 2:'preqin_location'})
couw_matched_df

,matched_sdc_couwriter,co_underwriter_match_score
0,bear stearns,51.851852
1,jp morgan,42.750000
2,jp morgan,42.750000
3,bear stearns,51.851852
4,bear stearns,56.842105
...,...,...
19643,None,NaN
19644,None,NaN
19645,None,NaN
19646,None,NaN


In [ ]:
# merge with the investor_df2 data
investor_df3 = pd.concat([investor_df2, couw_matched_df],axis=1)
investor_df3

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager,p_s_index1,matched_sdc_luwriter,l_underwriter_match_score,matched_sdc_couwriter,co_underwriter_match_score
0,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",0,deutsche banc alex brown,37.500000,bear stearns,51.851852
1,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",1,deutsche banc alex brown,29.268293,jp morgan,42.750000
2,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",2,deutsche banc alex brown,29.268293,jp morgan,42.750000
3,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",3,deutsche banc alex brown,37.500000,bear stearns,51.851852
4,questmark partners,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",4,deutsche banc alex brown,33.333333,bear stearns,56.842105
...,...,...,...,...,...,...,...,...
19643,woori venture partners,"[Shamus Group, Inc]",[],19643,shamus group,25.650000,None,NaN
19644,seoul technology investment,"[Shamus Group, Inc]",[],19644,shamus group,33.750000,None,NaN
19645,yuanta investment,"[Shamus Group, Inc]",[],19645,shamus group,19.655172,None,NaN
19646,gn tech venture capital,"[Shamus Group, Inc]",[],19646,shamus group,23.478261,None,NaN


In [ ]:
# make max score of underwriter matching
investor_df3['max_match_score'] = investor_df3.loc[:,['l_underwriter_match_score','co_underwriter_match_score']].max(axis = 1)

In [ ]:
investor_df3

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager,p_s_index1,matched_sdc_luwriter,l_underwriter_match_score,matched_sdc_couwriter,co_underwriter_match_score,max_match_score
0,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",0,deutsche banc alex brown,37.500000,bear stearns,51.851852,51.851852
1,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",1,deutsche banc alex brown,29.268293,jp morgan,42.750000,42.750000
2,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",2,deutsche banc alex brown,29.268293,jp morgan,42.750000,42.750000
3,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",3,deutsche banc alex brown,37.500000,bear stearns,51.851852,51.851852
4,questmark partners,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",4,deutsche banc alex brown,33.333333,bear stearns,56.842105,56.842105
...,...,...,...,...,...,...,...,...,...
19643,woori venture partners,"[Shamus Group, Inc]",[],19643,shamus group,25.650000,None,NaN,25.650000
19644,seoul technology investment,"[Shamus Group, Inc]",[],19644,shamus group,33.750000,None,NaN,33.750000
19645,yuanta investment,"[Shamus Group, Inc]",[],19645,shamus group,19.655172,None,NaN,19.655172
19646,gn tech venture capital,"[Shamus Group, Inc]",[],19646,shamus group,23.478261,None,NaN,23.478261


In [ ]:
# Need to add the date
preqin_sdc_merged_1024['deal_year'] = preqin_sdc_merged_1024['DEAL DATE'].dt.year
preqin_sdc_merged_1024['deal_year'].value_counts()

In [ ]:
investor_df4 = pd.concat([investor_df3, preqin_sdc_merged_1024.deal_year],axis=1)
investor_df4

,INVESTORS / BUYERS (FIRMS),lead_underwriter_list,CM_CL_comanager,p_s_index1,matched_sdc_luwriter,l_underwriter_match_score,matched_sdc_couwriter,co_underwriter_match_score,max_match_score,deal_year
0,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",0,deutsche banc alex brown,37.500000,bear stearns,51.851852,51.851852,1997
1,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",1,deutsche banc alex brown,29.268293,jp morgan,42.750000,42.750000,1998
2,domain associates,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",2,deutsche banc alex brown,29.268293,jp morgan,42.750000,42.750000,1999
3,kleiner perkins,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",3,deutsche banc alex brown,37.500000,bear stearns,51.851852,51.851852,1999
4,questmark partners,[Deutsche Banc Alex Brown],"[Bear Stearns & Co Inc, Fleet Boston Corp, JP ...",4,deutsche banc alex brown,33.333333,bear stearns,56.842105,56.842105,1999
...,...,...,...,...,...,...,...,...,...,...
19643,woori venture partners,"[Shamus Group, Inc]",[],19643,shamus group,25.650000,None,NaN,25.650000,2002
19644,seoul technology investment,"[Shamus Group, Inc]",[],19644,shamus group,33.750000,None,NaN,33.750000,2012
19645,yuanta investment,"[Shamus Group, Inc]",[],19645,shamus group,19.655172,None,NaN,19.655172,2017
19646,gn tech venture capital,"[Shamus Group, Inc]",[],19646,shamus group,23.478261,None,NaN,23.478261,2017


In [ ]:
investor_df4.to_csv('/content/drive/MyDrive/UT_research/VC_IB/results/preqin_sdc/investor_match_1025.csv')